This is still a testing workbook to demonstrate progress on the excel to scenario flows.

# Curves

In [ ]:
# For now a custom csv to pd df function - this will be handled by the reverse packer in the end

import pandas as pd
from pathlib import Path
from typing import Union

def read_curves_to_dataframe(
    curves_path: Union[str, Path],
    pattern: str = "*.csv",
    validate_length: bool = True
) -> pd.DataFrame:
    """
    Read multiple curve CSV files into a single DataFrame.

    Args:
        curves_path: Directory path containing the curve CSV files
        pattern: File pattern to match (default: "*.csv")
        validate_length: Whether to validate each curve has exactly 8760 values

    Returns:
        DataFrame with curves as columns, where column names are the curve keys
        (derived from filenames without extension)

    Raises:
        ValueError: If validation fails or files have issues
        FileNotFoundError: If no files found matching the pattern
    """
    curves_path = Path(curves_path)

    if not curves_path.exists():
        raise FileNotFoundError(f"Directory not found: {curves_path}")

    # Find all CSV files matching the pattern
    csv_files = list(curves_path.glob(pattern))

    if not csv_files:
        raise FileNotFoundError(f"No files found matching pattern '{pattern}' in {curves_path}")

    print(f"Found {len(csv_files)} curve files")

    curves_data = {}
    errors = []

    for csv_file in csv_files:
        # Use filename (without extension) as curve key, remove _curve suffix if present
        curve_key = csv_file.stem
        if curve_key.endswith('_curve'):
            curve_key = curve_key[:-6]  # Remove '_curve' suffix

        try:
            # Read CSV file - assuming single column of values, no headers
            curve_data = pd.read_csv(
                csv_file,
                header=None,           # No header row
                index_col=False,       # No index column
                dtype=float           # All values should be numeric
            )

            # Convert DataFrame to Series if single column
            if isinstance(curve_data, pd.DataFrame):
                if len(curve_data.columns) == 1:
                    curve_data = curve_data.iloc[:, 0]
                else:
                    errors.append(f"{curve_key}: Expected 1 column, found {len(curve_data.columns)}")
                    continue

            # Drop any NaN values
            curve_data = curve_data.dropna()

            # Validate length if requested
            if validate_length and len(curve_data) != 8760:
                errors.append(f"{curve_key}: Expected 8760 values, found {len(curve_data)}")
                continue

            # Store with curve key as column name
            curves_data[curve_key] = curve_data.values
            print(f"✓ Loaded curve '{curve_key}': {len(curve_data)} values")

        except Exception as e:
            errors.append(f"{curve_key}: Error reading file - {str(e)}")
            continue

    if errors:
        error_msg = "Errors reading curve files:\n" + "\n".join(f"  - {err}" for err in errors)
        if not curves_data:  # No curves loaded successfully
            raise ValueError(error_msg)
        else:
            print(f"Warning: Some curves failed to load:\n{error_msg}")

    if not curves_data:
        raise ValueError("No curves were successfully loaded")

    # Create DataFrame from the curves
    df = pd.DataFrame(curves_data)

    # Set index to represent hours (0-8759 for a full year)
    df.index.name = "hour"

    print(f"Created DataFrame with {len(df.columns)} curves and {len(df)} rows")
    return df

# User uploads Excel/CSV → DataFrame → CustomCurves object
from pyetm.models.custom_curves import CustomCurves

df = read_curves_to_dataframe("curve_examples/")
custom_curves = CustomCurves._from_dataframe(df)

In [ ]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()
scenario = Scenario.load(2690288)

# Update curves on scenario
scenario.update_custom_curves(custom_curves)